# Sigmoidal Fitting

In [ ]:
%%capture
%pip install matplotlib numpy scipy lmfit

## Input

Each line must contain either 2 or 3 fields separated by whitespace. 
An easy way is to simply copy/paste values from Excel.

Example:

    input_data = """
    <x-value>   <y-value>   [<weight>]
    <x-value>   <y-value>   [<weight>]
    <x-value>   <y-value>   [<weight>]
    [...]
    """

Note:
  - All lines must contain the same number of fields.
  - The first and last line must not be deleted.

In [ ]:
input_data = """
1	30.633	1
9	34.774	1
20	49.24	1
39	104	1
59	153.048	1
71	169.499	1
90	250.428	1
114.5	378.78	1
153	525.84	1
156	439.92	0.1
179	636.312	1
181	553.512	0.1
184	550	0.1
189.5	810	1
224	1080	1
"""

## Settings

In [ ]:
# fit in log10 space
logspace = True

# range of xaxis and fit prediction
xrange = [0, 280]

# figure width, height in inches
fig_size = 10, 6

# figure DPI
fig_dpi = 100

# Custom plot labels
fig_title = 'Sigmoidal fit'
fig_xaxis_label = 'x'
fig_yaxis_label = 'y'

## Calculation

In [ ]:
import math
import re
import sys

import numpy as np
import matplotlib.pyplot as plt

from io import StringIO
from lmfit import Model
from lmfit.models import PolynomialModel

plt.rcParams['figure.figsize'] = fig_size
plt.rcParams['figure.dpi'] = fig_dpi

In [ ]:
c = StringIO(input_data.strip())
data = np.loadtxt(c)
xx = data[:, 0]
yy = data[:, 1]

if logspace:
    yy = np.log10(yy)

try:
    weights = data[:, 2]
except IndexError:
    weights = None

if xrange is None:
    xrange = [np.min(xx), np.max(xx)]

In [ ]:
def sigmoid(x, s0, s1, s2, s3):
    return s0 + s1 / (1 + np.exp(-(x - s2) / s3))

model = Model(sigmoid)
params = model.make_params(
    s0=max(yy), s1=np.median(xx), s2=1, s3=min(yy),
)

result = model.fit(yy, params, x=xx, weights=weights)

xx_fit = np.linspace(xrange[0], xrange[-1], 100)
yy_fit = model.eval(result.params, x=xx_fit)

if logspace:
    yy = 10 ** yy
    yy_fit = 10 ** yy_fit

## Fit result

### Plot

In [ ]:
plt.plot(xx, yy, marker='o', label='Input data')
plt.plot(xx_fit, yy_fit, label='Fit')
plt.xlim(*xrange)
#plt.semilogy()
plt.grid()
plt.legend()
plt.suptitle(fig_title)
plt.xlabel(fig_xaxis_label)
plt.ylabel(fig_yaxis_label)
# plt.tight_layout()
plt.show()

### Report

In [ ]:
print(result.fit_report())

### Y-values

In [ ]:
yy_best = result.best_fit
if logspace:
    yy_best = 10 ** yy_best
# result_data = np.column_stack((xx, yy_best))
# np.savetxt(sys.stdout, result_data, fmt=['%g\t', '%.5f'], delimiter='')
result_data = yy_best
np.savetxt(sys.stdout, result_data, fmt='%.5f')
